In [1]:
import numpy as np
import plotly
from sklearn import linear_model
from sklearn import datasets

# Data Preparation

In [2]:
diabetes = datasets.load_diabetes()
x = diabetes.data
y_not_standardized = diabetes.target

In [ ]:
n = x.shape[0]
m = x.shape[1]
print(f"n={n}, m={m}")

n=442, m=10


x is already standardized. however y is not.

In [4]:
y = y_not_standardized - y_not_standardized.mean()
print(y_not_standardized.mean())
print(y.mean())

152.13348416289594
-7.716301202824617e-15


# Initialization

In [9]:
beta = np.zeros(m)
prediction = np.zeros(n)
residual = y.copy()
active_set = []
signs = np.zeros(m)
tol=1e-8
beta_path = [beta.copy()] 
prediction_path = [prediction.copy()]
active_sets = [active_set.copy()]


# Main loop

In [ ]:
for step in range(m):
    correlations = x.T @ residual
    c_max = np.max(np.abs(correlations))
    if c_max < tol:
        break
    candidates = np.where(np.abs(np.abs(correlations)-c_max) <= tol)[0] # can i do this better?
    for j in candidates:
        if j not in active_set:
            active_set.append(j)
            signs[j] = np.sign(correlations[j]) if correlations[j] != 0.0 else 1.0 # check this
    nump_active_set = np.array(active_set, dtype=int) # consider adding dype elsewhere
    active_signs = signs[nump_active_set]
    active_x = x[:, nump_active_set] * active_signs
    g_a = active_x.T @ active_x
    g_a_inverse = np.linalg.inv(g_a)
    ones_a = np.ones(len(nump_active_set))
    a_a = 1.0 / np.sqrt(ones_a.T @ g_a_inverse @ ones_a)
    w_a = a_a * (g_a_inverse @ ones_a)
    u_a = active_x @ w_a
    little_a = active_x.T @ u_a
    
    #find gamma step size
    gamma = np.inf
    j_star = None
    inactive = [j for j in range(m) if j not in active_set]
    for j in inactive:
        denominator_1 = a_a - little_a[j]
        if denominator_1 > tol:
            gamma1 = (c_max - correlations[j]) / denominator_1
            if gamma1 > tol and gamma1 < gamma:
                gamma = gamma1
                j_star = j
        denominator_2 = a_a + little_a[j]
        if denominator_2 > tol:
            gamma2 = (c_max + correlations[j]) / denominator_2
            if gamma2 > tol and gamma2 < gamma:
                gamma = gamma2
                j_star = j
    
    # update prediction and beta
    prediction += gamma * u_a
    for index, j in enumerate(nump_active_set):
        beta[j] += gamma * w_a[index] * signs[j]

    if j_star not in active_set: # do i need this condition?
        active_set.append(j_star)
        signs[j_star] = np.sign(correlations[j_star]) if correlations[j_star] != 0.0 else 1.0

    beta_path.append(beta.copy())
    prediction_path.append(prediction.copy())
    active_sets.append(active_set.copy())


    beta_path = np.vstack(beta_path)
    prediction_path = np.vstack(prediction_path)


print(beta_path)

IndexError: index 1 is out of bounds for axis 0 with size 1